# 4 - Cadena con FastAPI

<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/fastapi.png" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Cadena-de-LangChain" data-toc-modified-id="1---Cadena-de-LangChain-1">1 - Cadena de LangChain</a></span></li><li><span><a href="#2---Cadena-en-una-función" data-toc-modified-id="2---Cadena-en-una-función-2">2 - Cadena en una función</a></span></li><li><span><a href="#3---Código-de-FastAPI" data-toc-modified-id="3---Código-de-FastAPI-3">3 - Código de FastAPI</a></span></li></ul></div>

## 1 - Cadena de LangChain

Vamos a usar una cadena de Langchain sencilla para usarla dentro de una API de FastAPI. Vamos a ver otra vez la cadena paso a paso y crear una función con ella.

In [1]:
# primero cargamos la API KEY de OpenAI

from dotenv import load_dotenv 
import os

# carga de variables de entorno
load_dotenv()


# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
# preparamos el prompt

from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([
    
    ('system', '''Eres un historiador muy erudito que ofrece respuestas precisas y 
                  elocuentes a preguntas históricas y que responde en castellano.'''),
    
    ('human', '{pregunta}')
    
])

In [3]:
# iniciamos el modelo llm

from langchain_openai import ChatOpenAI

modelo = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

In [4]:
# parser de salida, transforma la salida a string

from langchain.schema import StrOutputParser

parser = StrOutputParser()

In [5]:
# creamos la cadena con lcel

cadena = prompt | modelo | parser

In [6]:
# llamada a la cadena

pregunta = '¿Cuales son las 7 maravillas del mundo?'

respuesta = cadena.invoke({'pregunta': pregunta})

In [7]:
respuesta

'Las Siete Maravillas del Mundo Antiguo son:\n\n1. La Gran Pirámide de Guiza en Egipto.\n2. Los Jardines Colgantes de Babilonia en Irak.\n3. La Estatua de Zeus en Olimpia, Grecia.\n4. El Templo de Artemisa en Éfeso, Turquía.\n5. El Mausoleo de Halicarnaso en Turquía.\n6. El Coloso de Rodas en la isla de Rodas, Grecia.\n7. El Faro de Alejandría en Egipto.\n\nEs importante mencionar que estas maravillas fueron seleccionadas en la antigüedad y que actualmente solo queda en pie la Gran Pirámide de Guiza.'

## 2 - Cadena en una función

Ahora que hemos visto esa cadena sencilla paso a paso, vamos a crear una función que reciba la pregunta del usuario y devuelva la respuesta del modelo. Esta función la usaremos después en una aplicación de FastAPI.

In [8]:
# importamos librerías

from dotenv import load_dotenv 
import os

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser



# carga de variables de entorno
load_dotenv()

# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



def cadena_llm(pregunta:str) -> str:
    
    """
    Esta función es para el uso de un LLM con una cadena sencilla:
    
    Params:
    pregunta: str, inoput del usuario
    
    Return:
    string, devuelve la pregunta del modelo
    """
    
    global OPENAI_API_KEY
    
    # preparamos el prompt
    prompt = ChatPromptTemplate.from_messages([

        ('system', '''Eres un historiador muy erudito que ofrece respuestas precisas y 
                      elocuentes a preguntas históricas y que responde en castellano.'''),

        ('human', '{pregunta}')

    ])


    # iniciamos el modelo llm
    modelo = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

    # parser de salida, transforma la salida a string
    parser = StrOutputParser()
    
    # creamos la cadena con lcel
    cadena = prompt | modelo | parser
    
    # llamada a la cadena
    respuesta = cadena.invoke({'pregunta': pregunta})
    
    return respuesta

In [9]:
cadena_llm(pregunta)

'Las Siete Maravillas del Mundo Antiguo son:\n\n1. La Gran Pirámide de Guiza en Egipto.\n2. Los Jardines Colgantes de Babilonia en Irak.\n3. La Estatua de Zeus en Olimpia, Grecia.\n4. El Templo de Artemisa en Éfeso, Turquía.\n5. El Mausoleo de Halicarnaso en Turquía.\n6. El Coloso de Rodas en la isla de Rodas, Grecia.\n7. El Faro de Alejandría en Egipto.\n\nEs importante mencionar que estas maravillas fueron seleccionadas en la antigüedad y que actualmente solo queda en pie la Gran Pirámide de Guiza.'

## 3 - Código de FastAPI

Ahora vamos a usar esta función para usarla a través de la API. En cuanto la aplicación realice la petición POST y reciba el mensaje del usuario, dicho mensaje se usará para crear la respuesta del modelo. El objetivo es crear un chat que podamos usar desde nuestra API.

In [5]:
# importamos librerías

from dotenv import load_dotenv 
import os

from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

from fastapi import FastAPI, Query



# carga de variables de entorno
load_dotenv()

# api key openai, nombre que tiene por defecto en LangChain
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


# iniciamos la aplicacion
app = FastAPI()

# almacén temporal de mensajes, en memoria
mensajes = []


def cadena_llm(pregunta:str) -> str:
    
    """
    Esta función es para el uso de un LLM con una cadena sencilla:
    
    Params:
    pregunta: str, inoput del usuario
    
    Return:
    string, devuelve la pregunta del modelo
    """
    
    global OPENAI_API_KEY
    
    # preparamos el prompt
    prompt = ChatPromptTemplate.from_messages([

        ('system', '''Eres un historiador muy erudito que ofrece respuestas precisas y 
                      elocuentes a preguntas históricas y que responde en castellano.'''),

        ('human', '{pregunta}')

    ])


    # iniciamos el modelo llm
    modelo = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

    # parser de salida, transforma la salida a string
    parser = StrOutputParser()
    
    # creamos la cadena con lcel
    cadena = prompt | modelo | parser
    
    # llamada a la cadena
    respuesta = cadena.invoke({'pregunta': pregunta})
    
    return respuesta



# endpoint para obtener todos los mensajes (GET)
@app.get('/mensajes')
def historial():
    
    # devuelve el json de mensajes
    return mensajes



# endpoint para el chat
@app.get('/chat')
def chat(query: str = Query(...)):
    
    global cadena_llm
    

    # respuesta del LLM
    respuesta = cadena_llm(query)

    # se añade a la lista
    mensajes.append({'nombre': 'Tu', 'mensaje': query})
    mensajes.append({'nombre': 'Bot', 'mensaje': respuesta})
    
    return {'nombre': 'Bot', 'mensaje': respuesta}
        

In [6]:
# ejecutar la app

import nest_asyncio
import uvicorn

if __name__ == '__main__':
    
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [13739]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50996 - "GET /chat?query=hola HTTP/1.1" 200 OK
INFO:     127.0.0.1:50997 - "GET /mensajes HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13739]
